# MOWNIT
## Lab 3
Jakub Karbowski

In [1]:
using Plots
using Interact
using DataFrames

WebIO._IJuliaInit()

# Warunki brzegowe
Węzły: $x_i, i\in \{1,\dots,n+1\}$
- Równość w węzłach (warunek zawsze stosowany):
    - $s'(x_1) = f(x_1)$
    - $s'(x_{n+1}) = f(x_{n+1})$
- Clamped left:
    - $s'(x_1) = f'(x_1)$
- Clamped right:
    - $s'(x_{n+1}) = f'(x_{n+1})$
- Periodic:
    - $s'(x_1) = s'(x_{n+1})$
- Natural left:
    - $s''(x_1) = 0$
- Natural right:
    - $s''(x_{n+1}) = 0$

In [2]:
@enum Boundary ClampedLeft ClampedRight Periodic NaturalLeft NaturalRight

# Pierwszego rzędu
Zero slotów na dodatkowe warunki brzegowe.
\begin{align}
    &s_i(x) = a_i + b_i(x-x_i) \\
    &\text{Środkowe węzły:} \\
    &\begin{cases}
        s_i(x_{i+1}) &= y_{i+1} \\
        s_i(x_{i+1}) &= s_{i+1}(x_{i+1})
    \end{cases} \\
    &\text{Węzły brzegowe:} \\
    &\begin{cases}
        s_1(x_1) &= y_1 \\
        s_n(x_{n+1}) &= y_{n+1}
    \end{cases}
\end{align}

In [3]:
function spline1(xs, ys, ys1, boundary)
    n = length(xs) - 1
    
    A = zeros(2n, 2n)
    Y = zeros(2n)
    
    # equations for middle nodes
    for i = 1:n-1
        # a_i + b_i * (x_{i+1} - x_i) = y_{i+1}
        A[2i, 2i-1:2i] = [ 1  xs[i+1]-xs[i] ]
        Y[2i] = ys[i+1]
        
        # a_i + b_i * (x_{i+1} - x_i) - a_{i+1} = 0
        A[2i+1, 2i-1:2i+1] = [ 1  xs[i+1]-xs[i]  -1 ]
        Y[2i+1] = 0
    end
    
    # left boundary
    # a_1 = y_1
    A[1, 1] = 1
    Y[1] = ys[1]
    
    # right boundary
    # a_n + b_n * (x_{n+1} - x_n) = y_{n+1}
    A[2n, 2n-1:2n] = [ 1  xs[n+1]-xs[n] ]
    Y[2n] = ys[n+1]
    
    X = A \ Y
    
    function (x)
        for i = 1:n
            if xs[i] <= x <= xs[i+1]
                return X[2i-1] + X[2i] * (x - xs[i])
            end
        end
    end
end

spline1 (generic function with 1 method)

# Drugiego rzędu
Jeden slot na dodatkowy warunek brzegowy.
\begin{align}
    &s_i(x) = a_i + b_i(x-x_i) + c_i(x-x_i)^2 \\
    &\text{Środkowe węzły:} \\
    &\begin{cases}
        s_i(x_{i+1}) &= y_{i+1} \\
        s_i(x_{i+1}) &= s_{i+1}(x_{i+1}) \\
        s'_i(x_{i+1}) &= s'_{i+1}(x_{i+1})
    \end{cases} \\
    &\text{Węzły brzegowe:} \\
    &\begin{cases}
        s_1(x_1) &= y_1 \\
        s_n(x_{n+1}) &= y_{n+1}
    \end{cases} \\
    &\text{Clamped left:} \\
    &s'_1(x_1) = y'_1 \\
    &\text{Clamped right:} \\
    &s'_n(x_{n+1}) = y'_{n+1} \\
    &\text{Periodic:} \\
    &s'_1(x_1) = s'_n(x_{n+1})
\end{align}

In [4]:
function spline2(xs, ys, ys1, boundary)
    n = length(xs) - 1
    
    A = zeros(3n, 3n)
    Y = zeros(3n)
    
    # equations for middle nodes
    for i = 1:n-1
        A[3i-1, 3i-2:3i] = [ 1  xs[i+1]-xs[i]  (xs[i+1]-xs[i])^2 ]
        Y[3i-1] = ys[i+1]
        
        A[3i, 3i-2:3i+1] = [ 1  xs[i+1]-xs[i]  (xs[i+1]-xs[i])^2  -1 ]
        Y[3i] = 0
        
        A[3i+1, 3i-1:3i+2] = [ 1  2*(xs[i+1]-xs[i])  0  -1 ]
        Y[3i+1] = 0
    end
    
    # left boundary
    A[1, 1] = 1
    Y[1] = ys[1]
    
    # right boundary
    A[3n-1, 3n-2:3n] = [ 1  xs[n+1]-xs[n]  (xs[n+1]-xs[n])^2 ]
    Y[3n-1] = ys[n+1]
    
    if ClampedLeft in boundary
        A[3n, 2] = 1
        Y[3n] = ys1[1]
    elseif ClampedRight in boundary
        A[3n, 3n-1:3n] = [ 1  2*(xs[n+1]-xs[n]) ]
        Y[3n] = ys1[n+1]
    elseif Periodic in boundary
        A[3n, 2] = -1
        A[3n, 3n-1:3n] = [ 1  2*(xs[n+1]-xs[n]) ]
        Y[3n] = 0
    end
    
    X = A \ Y
    
    function (x)
        for i = 1:n
            if xs[i] <= x <= xs[i+1]
                return X[3i-2] + X[3i-1] * (x - xs[i]) + X[3i] * (x - xs[i])^2
            end
        end
    end
end

spline2 (generic function with 1 method)

# Trzeciego rzędu
Dwa sloty na dodatkowe warunki brzegowe.
\begin{align}
    &s_i(x) = a_i + b_i(x-x_i) + c_i(x-x_i)^2 + d_i(x-x_i)^3 \\
    &\text{Środkowe węzły:} \\
    &\begin{cases}
        s_i(x_{i+1}) &= y_{i+1} \\
        s_i(x_{i+1}) &= s_{i+1}(x_{i+1}) \\
        s'_i(x_{i+1}) &= s'_{i+1}(x_{i+1}) \\
        s''_i(x_{i+1}) &= s''_{i+1}(x_{i+1})
    \end{cases} \\
    &\text{Węzły brzegowe:} \\
    &\begin{cases}
        s_1(x_1) &= y_1 \\
        s_n(x_{n+1}) &= y_{n+1}
    \end{cases} \\
    &\text{Clamped left:} \\
    &s'_1(x_1) = y'_1 \\
    &\text{Clamped right:} \\
    &s'_n(x_{n+1}) = y'_{n+1} \\
    &\text{Periodic:} \\
    &s'_1(x_1) = s'_n(x_{n+1}) \\
    &\text{NaturalLeft:} \\
    &s''_1(x_1) = 0 \\
    &\text{NaturalRight:} \\
    &s''_n(x_{n+1}) = 0
\end{align}

In [5]:
function spline3(xs, ys, ys1, boundary)
    n = length(xs) - 1
    
    A = zeros(4n, 4n)
    Y = zeros(4n)
    
    # equations for middle nodes
    for i = 1:n-1
        dx = xs[i+1] - xs[i]
        
        A[4i-3, 4i-3:4i] = [ 1 dx dx^2 dx^3 ]
        Y[4i-3] = ys[i+1]
        
        A[4i-2, 4i-3:4i+1] = [ 1 dx dx^2 dx^3 -1 ]
        Y[4i-2] = 0
        
        A[4i-1, 4i-2:4i+2] = [ 1 2dx 3dx^2 0 -1 ]
        Y[4i-1] = 0
        
        A[4i, 4i-1:4i+3] = [ 1 3dx 0 0 -1 ]
        Y[4i] = 0
    end
    
    # left boundary
    A[4n-3, 1] = 1
    Y[4n-3] = ys[1]

    # right boundary
    dx = xs[n+1] - xs[n]
    A[4n-2, 4n-3:4n] = [ 1 dx dx^2 dx^3 ]
    Y[4n-2] = ys[n+1]
    
    usedslots = 0
    
    if ClampedLeft in boundary
        A[4n-1+usedslots, 2] = 1
        Y[4n-1+usedslots] = ys1[1]
        usedslots += 1
    end

    if ClampedRight in boundary
        A[4n-1+usedslots, 4n-2:4n] = [ 1 2dx 3dx^2 ]
        Y[4n-1+usedslots] = ys1[n+1]
        usedslots += 1
    end
    
    if Periodic in boundary
        A[4n-1+usedslots, 2] = -1
        A[4n-1+usedslots, 4n-2:4n] = [ 1 2dx 3dx^2 ]
        Y[4n-1+usedslots] = 0
        usedslots += 1
    end
    
    if NaturalLeft in boundary
        A[4n-1+usedslots, 3] = 2
        Y[4n-1+usedslots] = 0
        usedslots += 1
    end
    
    if NaturalRight in boundary
        A[4n-1+usedslots, 4n-1:4n] = [ 2 6dx ]
        Y[4n-1+usedslots] = 0
        usedslots += 1
    end
    
    X = A \ Y
    
    function (x)
        for i = 1:n
            if xs[i] <= x <= xs[i+1]
                return X[4i-3] + X[4i-2] * (x - xs[i]) + X[4i-1] * (x - xs[i])^2 + X[4i] * (x - xs[i])^3
            end
        end
    end
end

spline3 (generic function with 1 method)

In [6]:
sqerr(f, g, x) = sum( @. (f(x) - g(x))^2 )

sqerr (generic function with 1 method)

In [7]:
k = 2
m = 1

# f(x) = exp(-k*sin(m*x))+k*cos(m*x) + 1
# df(x) = k*m*(cos(m*x)*(-exp(-k*sin(m*x))) - sin(m*x))
f(x) = atan(x)
df(x) = 1/(1+x^2)

xlo = -10
xhi = 10
xs = range(xlo, xhi, length=1000)

-10.0:0.02002002002002002:10.0

In [8]:
@manipulate for n=2:50,
                spline=[spline1, spline2, spline3],
                clampedleft=false,
                clampedright=false,
                periodic=false,
                naturalleft=false,
                naturalright=false
    nodes = range(xlo, xhi, length=n)

    boundary = Boundary[]
    clampedleft && push!(boundary, ClampedLeft)
    clampedright && push!(boundary, ClampedRight)
    periodic && push!(boundary, Periodic)
    naturalleft && push!(boundary, NaturalLeft)
    naturalright && push!(boundary, NaturalRight)
    
    s = try
        spline(nodes, f.(nodes), df.(nodes), boundary)
    catch
        x -> 0
    end
    
    plot(
        xs,
        f.(xs),
        label="f(x)",
        legend=:outerbottom,
        line=:dash,
        title="$boundary",
    )
    
    scatter!(
        nodes,
        f.(nodes),
        label="Nodes (n=$n)",
    )
    
    plot!(
        xs,
        s.(xs),
        label="$spline",
    )
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["n"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol, Any}(:max => 50, :min => 2, :attributes => Dict{Any, Any}(:type => "range", Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}", "orient" => "horizontal"), :step => 1, :className => "slider slider is-fullwidth", :style => Dict{Any, Any}()))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{Observables.AbstractObservable, Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing), "index" => (Observable{Int64} with 2 listeners. Value:
26, nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/Users/carbon/.julia/packages/Knockout/3gjS1/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/carbon/.julia/packages/Knockout/3gjS1/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/carbon/.julia/packages/InteractBase/h8Fwe/src/../assets/all.js"), Asset("css", nothing, "/Users/carbon/.julia/packages/InteractBase/h8Fwe/src/../assets/style.css"), Asset("css", nothing, "/Users/carbon/.julia/packages/Interact/pVyN2/src/../assets/bulma_confined.min.css")], Dict{Any, Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Condition(Base.InvasiveLinkedList{Task}(Task (runnable) @0x00000001168f2230, Task (runnable) @0x00000001168f2230), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"2\",\"3\",\"4\",\"5\",\"6\",\"7\",\"8\",\"9\",\"10\",\"11\",\"12\",\"13\",\"14\",\"15\",\"16\",\"17\",\"18\",\"19\",\"20\",\"21\",\"22\",\"23\",\"24\",\"25\",\"26\",\"27\",\"28\",\"29\",\"30\",\"31\",\"32\",\"33\",\"34\",\"35\",\"36\",\"37\",\"38\",\"39\",\"40\",\"41\

# Tabelki

In [9]:
function trial(n, titles, params)
    dferr = DataFrame(n = [])
    [insertcols!(dferr, Symbol(t)=>[]) for t=titles]
    for i = n
        row = Real[i]
        for (spline, boundary) = params
            nodes = range(xlo, xhi, length=i)

            s = spline(nodes, f.(nodes), df.(nodes), boundary)
            err = sqerr(f, s, xs)
            push!(row, err)
        end
        push!(dferr, row)
    end
    dferr
end

trial (generic function with 1 method)

# Clamped vs Natural (3rd)
Clamped 3rd lepsze.

In [10]:
trial(10:10:100, ["Clamped", "Natural"], [
    (spline3, [ClampedLeft, ClampedRight]),
    (spline3, [NaturalLeft, NaturalRight]),
])

,n,Clamped,Natural
,Any,Any,Any
1,10,0.544813,0.545902
2,20,0.000240081,0.000240691
3,30,0.000214065,0.000214139
4,40,4.10962e-5,4.11133e-5
5,50,6.24331e-6,6.24876e-6
6,60,1.10788e-6,1.11004e-6
7,70,2.51751e-7,2.52738e-7
8,80,7.19016e-8,7.24039e-8
9,90,2.45684e-8,2.48453e-8


# Clamped vs Periodic (2nd)
Periodic 2nd lepsze.

In [11]:
trial(10:10:100, ["Clamped left", "Clamped right", "Periodic"], [
    (spline2, [ClampedLeft]),
    (spline2, [ClampedRight]),
    (spline2, [Periodic]),
])

,n,Clamped left,Clamped right,Periodic
,Any,Any,Any,Any
1,10,56.6795,56.6795,26.3772
2,20,2.6406,2.6406,1.27995
3,30,0.117885,0.117885,0.0590662
4,40,0.00538891,0.00538891,0.00284126
5,50,0.000277194,0.000277194,0.00016766
6,60,2.31857e-5,2.31857e-5,1.83877e-5
7,70,4.66822e-6,4.66822e-6,4.46903e-6
8,80,1.65901e-6,1.65901e-6,1.64926e-6
9,90,7.28795e-7,7.28795e-7,7.28525e-7


# Periodic 2nd vs Clamped 3rd
Clamped 3rd lepsze.

In [12]:
trial(10:10:100, ["Periodic 2nd", "Clamped 3rd"], [
    (spline2, [Periodic]),
    (spline3, [ClampedLeft, ClampedRight]),
])

,n,Periodic 2nd,Clamped 3rd
,Any,Any,Any
1,10,26.3772,0.544813
2,20,1.27995,0.000240081
3,30,0.0590662,0.000214065
4,40,0.00284126,4.10962e-5
5,50,0.00016766,6.24331e-6
6,60,1.83877e-5,1.10788e-6
7,70,4.46903e-6,2.51751e-7
8,80,1.64926e-6,7.19016e-8
9,90,7.28525e-7,2.45684e-8
